#Install and Load Library Needed

In [ ]:
!pip install segmentation-models-3d

In [ ]:
from keras.models import load_model
import nibabel as nib
import numpy as np
import random
import matplotlib.pyplot as plt
import segmentation_models_3D as sm
import tensorflow as tf
import re
import os

Segmentation Models: using `tf.keras` framework.


#Define the costumized metrics from segmentation library

In [ ]:
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

#Lungs Models

##Load Test Data for Lungs Evaluation

In [ ]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import random
import tifffile as tiff
from tifffile import imread

def load_img(img_dir, img_list):
    images=[]
    for i, image_name in enumerate(img_list):    
        if (image_name.split('.')[1] == 'npy'):
            
            image = np.load(img_dir+image_name).astype(np.float32)
                      
            images.append(image)
    images = np.array(images)
    
    return(images)

  
def imageLoader(img_dir, img_list, mask_dir, mask_list, batch_size):
  
  length = len(img_list)

  while True:
    batch_start = 0
    batch_end = batch_size

    while batch_start < length:
      limit = min(batch_end, length)
      X = load_img(img_dir, img_list[batch_start:limit])
      Y = load_img(mask_dir, mask_list[batch_start:limit])

      yield(X, Y)

      batch_start += batch_size
      batch_end += batch_size


In [ ]:
train_img_dir = '/content/drive/MyDrive/3D Segmentation/splitted 75 lung 128 data/train/ctscan/'
train_mask_dir = '/content/drive/MyDrive/3D Segmentation/splitted 75 lung 128 data/train/lung/'

train_img_list= sorted_alphanumeric(os.listdir(train_img_dir))
train_mask_list = sorted_alphanumeric(os.listdir(train_mask_dir))

In [ ]:
batch_size = 1

train_img_datagen = imageLoader(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list, batch_size)

In [ ]:
test_img_dir = "/content/drive/MyDrive/3D Segmentation/splitted 75 lung 128 data/val/ctscan/"
test_mask_dir = "/content/drive/MyDrive/3D Segmentation/splitted 75 lung 128 data/val/lung/"

test_img_list= sorted_alphanumeric(os.listdir(test_img_dir))
test_mask_list = sorted_alphanumeric(os.listdir(test_mask_dir))

In [ ]:
batch_size = 1

test_img_datagen = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, batch_size)

In [ ]:
steps_per_epoch = len(train_img_list)//batch_size
val_steps_per_epoch = len(test_img_list)//batch_size

In [ ]:
imgtest, msktest = test_img_datagen.__next__()

##Pure 3DUnet

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/Lungs model/pureunetlung75.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'binary_focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 33s 1s/step - loss: 0.0218 - iou_score: 0.9627 - dice_loss: 0.0194 - jaccard_loss: 0.0380 - accuracy: 0.9961 - precision: 0.9821 - recall: 0.9816 - val_loss: 0.0352 - val_iou_score: 0.9415 - val_dice_loss: 0.0304 - val_jaccard_loss: 0.0589 - val_accuracy: 0.9937 - val_precision: 0.9758 - val_recall: 0.9657


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 1s 540ms/step - loss: 0.0343 - iou_score: 0.9432 - dice_loss: 0.0295 - jaccard_loss: 0.0573 - accuracy: 0.9937 - precision: 0.9758 - recall: 0.9657


[0.03428349643945694,
 0.9431798458099365,
 0.029482901096343994,
 0.057276904582977295,
 0.9937433004379272,
 0.9758269786834717,
 0.9657437801361084]

##3DVGGUnet (VGG19)

### **Without** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/Lungs model/vggnew/vgg19unetlung75_notf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 35s 2s/step - loss: 0.0058 - iou_score: 0.9911 - dice_loss: 0.0055 - jaccard_loss: 0.0110 - accuracy: 0.9991 - precision_1: 0.9945 - recall_1: 0.9967 - val_loss: 0.0420 - val_iou_score: 0.9309 - val_dice_loss: 0.0367 - val_jaccard_loss: 0.0704 - val_accuracy: 0.9920 - val_precision_1: 0.9555 - val_recall_1: 0.9707


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 7s 7s/step - loss: 0.0430 - iou_score: 0.9287 - dice_loss: 0.0376 - jaccard_loss: 0.0726 - accuracy: 0.9920 - precision_1: 0.9555 - recall_1: 0.9707


[0.04295634105801582,
 0.9287487864494324,
 0.037646710872650146,
 0.07256132364273071,
 0.9919915199279785,
 0.9555427432060242,
 0.9706929922103882]

### **With** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/Lungs model/vggnew/vgg19unetlung75_tf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 30s 2s/step - loss: 0.0105 - iou_score: 0.9821 - dice_loss: 0.0099 - jaccard_loss: 0.0195 - accuracy: 0.9981 - precision_2: 0.9886 - recall_2: 0.9933 - val_loss: 0.0398 - val_iou_score: 0.9393 - val_dice_loss: 0.0320 - val_jaccard_loss: 0.0618 - val_accuracy: 0.9934 - val_precision_2: 0.9777 - val_recall_2: 0.9600


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 3s 3s/step - loss: 0.0396 - iou_score: 0.9395 - dice_loss: 0.0318 - jaccard_loss: 0.0616 - accuracy: 0.9934 - precision_2: 0.9777 - recall_2: 0.9600


[0.039649177342653275,
 0.9394885301589966,
 0.031767845153808594,
 0.06157946586608887,
 0.9933502078056335,
 0.9777266979217529,
 0.9600354433059692]

##3DResUnet (ResNet152)

### **Without** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/Lungs model/resenet152unetlung75_notf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 40s 1s/step - loss: 0.0068 - iou_score: 0.9894 - dice_loss: 0.0065 - jaccard_loss: 0.0130 - accuracy: 0.9989 - precision_3: 0.9937 - recall_3: 0.9960 - val_loss: 0.0942 - val_iou_score: 0.8482 - val_dice_loss: 0.0861 - val_jaccard_loss: 0.1542 - val_accuracy: 0.9793 - val_precision_3: 0.8662 - val_recall_3: 0.9554


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 4s 4s/step - loss: 0.1010 - iou_score: 0.8325 - dice_loss: 0.0928 - jaccard_loss: 0.1698 - accuracy: 0.9793 - precision_3: 0.8662 - recall_3: 0.9554


[0.10098471492528915,
 0.8325499892234802,
 0.09280425310134888,
 0.16984599828720093,
 0.9793353080749512,
 0.8662249445915222,
 0.955388605594635]

### **With** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/Lungs model/resenet152unetlung75_tf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 36s 1s/step - loss: 0.0048 - iou_score: 0.9925 - dice_loss: 0.0046 - jaccard_loss: 0.0091 - accuracy: 0.9992 - precision_4: 0.9955 - recall_4: 0.9973 - val_loss: 0.0557 - val_iou_score: 0.9163 - val_dice_loss: 0.0444 - val_jaccard_loss: 0.0848 - val_accuracy: 0.9909 - val_precision_4: 0.9654 - val_recall_4: 0.9496


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 4s 4s/step - loss: 0.0544 - iou_score: 0.9183 - dice_loss: 0.0431 - jaccard_loss: 0.0827 - accuracy: 0.9909 - precision_4: 0.9654 - recall_4: 0.9496


[0.05436595529317856,
 0.9183058142662048,
 0.043125033378601074,
 0.08268451690673828,
 0.9909151792526245,
 0.9653641581535339,
 0.9495924711227417]

##3DDenseUnet (DenseNet201)

### **Without** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/Lungs model/densenet201unetlung75_notf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 40s 1s/step - loss: 4.4323e-04 - iou_score: 0.9994 - dice_loss: 4.3369e-04 - jaccard_loss: 8.6669e-04 - accuracy: 0.9999 - precision_5: 0.9995 - recall_5: 1.0000 - val_loss: 0.0572 - val_iou_score: 0.9170 - val_dice_loss: 0.0441 - val_jaccard_loss: 0.0840 - val_accuracy: 0.9912 - val_precision_5: 0.9762 - val_recall_5: 0.9415


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 6s 6s/step - loss: 0.0551 - iou_score: 0.9204 - dice_loss: 0.0420 - jaccard_loss: 0.0806 - accuracy: 0.9912 - precision_5: 0.9762 - recall_5: 0.9415


[0.05506645515561104,
 0.9203735589981079,
 0.04198354482650757,
 0.08058393001556396,
 0.9912405014038086,
 0.9762193560600281,
 0.9414817690849304]

### **With** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/Lungs model/densenet201unetlung75_tf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 39s 1s/step - loss: 0.0024 - iou_score: 0.9970 - dice_loss: 0.0024 - jaccard_loss: 0.0047 - accuracy: 0.9997 - precision_6: 0.9982 - recall_6: 0.9989 - val_loss: 0.0511 - val_iou_score: 0.9242 - val_dice_loss: 0.0402 - val_jaccard_loss: 0.0769 - val_accuracy: 0.9919 - val_precision_6: 0.9764 - val_recall_6: 0.9472


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 4s 4s/step - loss: 0.0500 - iou_score: 0.9260 - dice_loss: 0.0390 - jaccard_loss: 0.0751 - accuracy: 0.9919 - precision_6: 0.9764 - recall_6: 0.9472


[0.04998437315225601,
 0.9259830117225647,
 0.039022743701934814,
 0.07511430978775024,
 0.991857647895813,
 0.9763652682304382,
 0.9472149610519409]

#Lungs and Infections Models

##Load Test Data for Lungs and Infections Evaluation

In [ ]:
train_img_dir = '/content/drive/MyDrive/3D Segmentation/splitted 75 lim 128 data/train/ctscan/'
train_mask_dir = '/content/drive/MyDrive/3D Segmentation/splitted 75 lim 128 data/train/lim/'

train_img_list= sorted_alphanumeric(os.listdir(train_img_dir))
train_mask_list = sorted_alphanumeric(os.listdir(train_mask_dir))

In [ ]:
batch_size = 1

train_img_datagen = imageLoader(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list, batch_size)

In [ ]:
test_img_dir = "/content/drive/MyDrive/3D Segmentation/splitted 75 lim 128 data/val/ctscan/"
test_mask_dir = "/content/drive/MyDrive/3D Segmentation/splitted 75 lim 128 data/val/lim/"

test_img_list= sorted_alphanumeric(os.listdir(test_img_dir))
test_mask_list = sorted_alphanumeric(os.listdir(test_mask_dir))

In [ ]:
batch_size = 1

test_img_datagen = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, batch_size)

In [ ]:
imgtest, msktest = test_img_datagen.__next__()

##Pure 3DUNET

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/lim model/pureunetlim75.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 11s 617ms/step - loss: 0.0807 - iou_score: 0.8808 - dice_loss: 0.0780 - jaccard_loss: 0.1208 - accuracy: 0.9937 - precision_9: 0.9937 - recall_9: 0.9937 - val_loss: 0.1245 - val_iou_score: 0.8179 - val_dice_loss: 0.1172 - val_jaccard_loss: 0.1835 - val_accuracy: 0.9878 - val_precision_9: 0.9878 - val_recall_9: 0.9878


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 1s 1s/step - loss: 0.1212 - iou_score: 0.8158 - dice_loss: 0.1139 - jaccard_loss: 0.1855 - accuracy: 0.9878 - precision_9: 0.9878 - recall_9: 0.9878


[0.12115620076656342,
 0.8158137202262878,
 0.11389273405075073,
 0.18554049730300903,
 0.9877713918685913,
 0.9877756834030151,
 0.9877675175666809]

##3DVGGUNET (VGG19)

### **Without** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/lim model/vggnew/vgg19unetlim75_notf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 33s 2s/step - loss: 0.0634 - iou_score: 0.9090 - dice_loss: 0.0619 - jaccard_loss: 0.0956 - accuracy: 0.9947 - precision_10: 0.9947 - recall_10: 0.9947 - val_loss: 0.1853 - val_iou_score: 0.7495 - val_dice_loss: 0.1755 - val_jaccard_loss: 0.2527 - val_accuracy: 0.9839 - val_precision_10: 0.9839 - val_recall_10: 0.9838


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 3s 3s/step - loss: 0.2049 - iou_score: 0.7276 - dice_loss: 0.1951 - jaccard_loss: 0.2739 - accuracy: 0.9839 - precision_10: 0.9839 - recall_10: 0.9838


[0.20491833984851837,
 0.7276301980018616,
 0.19508790969848633,
 0.2739112377166748,
 0.983858585357666,
 0.9838941097259521,
 0.9838374853134155]

### **With** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/lim model/vggnew/vgg19unetlim75_tf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 30s 2s/step - loss: 0.0388 - iou_score: 0.9506 - dice_loss: 0.0379 - jaccard_loss: 0.0526 - accuracy: 0.9971 - precision_11: 0.9971 - recall_11: 0.9971 - val_loss: 0.1741 - val_iou_score: 0.7608 - val_dice_loss: 0.1640 - val_jaccard_loss: 0.2419 - val_accuracy: 0.9840 - val_precision_11: 0.9840 - val_recall_11: 0.9839


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 3s 3s/step - loss: 0.1979 - iou_score: 0.7340 - dice_loss: 0.1878 - jaccard_loss: 0.2676 - accuracy: 0.9840 - precision_11: 0.9840 - recall_11: 0.9839


[0.19785478711128235,
 0.7340170741081238,
 0.1877616047859192,
 0.2675795555114746,
 0.9839593172073364,
 0.9839940667152405,
 0.9839392900466919]

##3DRESUNET (ResNet152)

### **Without** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/lim model/resnet15unetlim75_notf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 46s 1s/step - loss: 0.0471 - iou_score: 0.9364 - dice_loss: 0.0460 - jaccard_loss: 0.0676 - accuracy: 0.9965 - precision_2: 0.9965 - recall_2: 0.9965 - val_loss: 0.2023 - val_iou_score: 0.7348 - val_dice_loss: 0.1911 - val_jaccard_loss: 0.2678 - val_accuracy: 0.9833 - val_precision_2: 0.9833 - val_recall_2: 0.9833


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 6s 6s/step - loss: 0.2273 - iou_score: 0.7089 - dice_loss: 0.2161 - jaccard_loss: 0.2924 - accuracy: 0.9833 - precision_2: 0.9833 - recall_2: 0.9833


[0.22728726267814636,
 0.7088800072669983,
 0.2161272168159485,
 0.292374849319458,
 0.9833076596260071,
 0.9833407998085022,
 0.9832866787910461]

### **With** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/lim model/resnet15unetlim75_tf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 36s 1s/step - loss: 0.0431 - iou_score: 0.9436 - dice_loss: 0.0423 - jaccard_loss: 0.0607 - accuracy: 0.9969 - precision_3: 0.9970 - recall_3: 0.9969 - val_loss: 0.1841 - val_iou_score: 0.7498 - val_dice_loss: 0.1729 - val_jaccard_loss: 0.2524 - val_accuracy: 0.9832 - val_precision_3: 0.9832 - val_recall_3: 0.9831


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 4s 4s/step - loss: 0.1935 - iou_score: 0.7381 - dice_loss: 0.1822 - jaccard_loss: 0.2637 - accuracy: 0.9832 - precision_3: 0.9832 - recall_3: 0.9831


[0.19345742464065552,
 0.738065242767334,
 0.18223971128463745,
 0.26374274492263794,
 0.9831730723381042,
 0.983208954334259,
 0.9831493496894836]

##3DDenseUNET (DenseNet201)

### **Without** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/lim model/densenet201unetlim75_notf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - ETA: 0s - loss: 0.0506 - iou_score: 0.9312 - dice_loss: 0.0495 - jaccard_loss: 0.0738 - accuracy: 0.9962 - precision_4: 0.9962 - recall_4: 0.9962WARNING:tensorflow:6 out of the last 14 calls to <function Model.make_test_function.<locals>.test_function at 0x7f931fcc6d40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


15/15 [==============================] - 42s 1s/step - loss: 0.0506 - iou_score: 0.9312 - dice_loss: 0.0495 - jaccard_loss: 0.0738 - accuracy: 0.9962 - precision_4: 0.9962 - recall_4: 0.9962 - val_loss: 0.2100 - val_iou_score: 0.7260 - val_dice_loss: 0.1988 - val_jaccard_loss: 0.2765 - val_accuracy: 0.9825 - val_precision_4: 0.9826 - val_recall_4: 0.9825


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 5s 5s/step - loss: 0.2196 - iou_score: 0.7143 - dice_loss: 0.2084 - jaccard_loss: 0.2875 - accuracy: 0.9825 - precision_4: 0.9826 - recall_4: 0.9825


[0.219605952501297,
 0.7143351435661316,
 0.2084028124809265,
 0.2874996066093445,
 0.9825249910354614,
 0.9825748205184937,
 0.982498049736023]

### **With** Transfer Learning

In [ ]:
model = load_model('/content/drive/MyDrive/3D Segmentation/lim model/densenet201unetlim75_tf.h5', custom_objects = {'dice_loss_plus_1focal_loss': total_loss, 'iou_score':sm.metrics.IOUScore(threshold=0.5), 'f1-score':sm.metrics.FScore(),'dice_loss':sm.losses.DiceLoss(),'focal_loss':sm.losses.CategoricalFocalLoss(),'jaccard_loss':sm.losses.JaccardLoss()})

In [ ]:
#add more metrics
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.losses.DiceLoss(), sm.losses.JaccardLoss(),'accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)

mevaluate = model.fit(train_img_datagen,
                      steps_per_epoch=steps_per_epoch,
                      epochs=1,
                      validation_data=test_img_datagen,
                      validation_steps=val_steps_per_epoch)

15/15 [==============================] - 40s 1s/step - loss: 0.0521 - iou_score: 0.9281 - dice_loss: 0.0508 - jaccard_loss: 0.0762 - accuracy: 0.9956 - precision_5: 0.9956 - recall_5: 0.9956 - val_loss: 0.1839 - val_iou_score: 0.7522 - val_dice_loss: 0.1722 - val_jaccard_loss: 0.2495 - val_accuracy: 0.9835 - val_precision_5: 0.9836 - val_recall_5: 0.9835


In [ ]:
test = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, 5)

imgtest, msktest = test.__next__()

In [ ]:
model.evaluate(imgtest,msktest)

1/1 [==============================] - 6s 6s/step - loss: 0.2157 - iou_score: 0.7193 - dice_loss: 0.2040 - jaccard_loss: 0.2818 - accuracy: 0.9835 - precision_5: 0.9836 - recall_5: 0.9835


[0.2156600058078766,
 0.7192549705505371,
 0.2039770483970642,
 0.28177642822265625,
 0.9835405349731445,
 0.9835726022720337,
 0.9835225939750671]